# IoT Kernel

The *IoT Kernel* is a specialized Jupyter kernel that evaluates MicroPython code on connected microcontrollers. Features include:

* discover and connect to wired and wireless microcontrollers (`%discover` and `%connect`)
* synchronize files with the host (`%rsync`)
* softreset (`%softreset`)
* evaluate code on the host (`%%host`)
* exchange variables between microcontrollers and the host (`%store`)
* access the shell (`%%bash`)
* access other services of *ide49*, e.g. cross-compilers (`%%service`)

## Magics

Many of these features are accessed via magics. `%lsmagic` lists available magics.

In [1]:
%lsmagic

Line Magic:    -h shows help (e.g. %discover -h)
  %cat         Print contents of named file on microcontroller
  %cd          Change current working directory on host.
  %connect     Connect to device
  %cp          Copy files between host and microcontroller.
  %discover    Discover available devices
  %gettime     Query microcontroller time
  %loglevel    Set logging level.
  %lsmagic     List all magic functions.
  %mkdirs      Create all directories specified by the path, as needed.
  %name        Name of currently connected microcontroller.
  %pip         Install packages from PyPi
  %platform    sys.platform of currently connected device.
  %rdiff       Show differences between microcontroller and host directories
  %register    Register device
  %rlist       List files on microcontroller
  %rm          Delete files relative to path.
  %rsync       Synchronize microcontroller to host directories
  %softreset   Reset microcontroller. Similar to pressing the reset button.
  %store

The `-h` option on any magic returns additional information. E.g.

In [5]:
%discover -h

usage: %discover [-h] [-a] [-v]

Discover available devices

optional arguments:
  -h, --help     show this help message and exit
  -a, --all      list all devices connected to USB ports
  -v, --verbose  show uid


In [1]:
%%host -h

usage: %%host [-h]

Pass cell to host (cPython) for evaluation.

optional arguments:
  -h, --help  show this help message and exit


## MicroPython

At any one time, the kernel is connected to one (or no) microcontroller. Code typed into Jupyter cells is sent to the "connected" microcontroller for evaluation. The `%discover` magic scans USB ports and listens for advertisements from available devices without hunting down USB port names or IP addresses. For each device to which connection is successful, the magic reports the device name, URL, and, optionally, UID (`-v` flag). Device names are specified in [configuration files](#device-configuration)).

**Note:** Use a powered USB hub to reliably connect more than one microcontroller (especially power hungry ESP32's) to a Raspberry Pi!

In [7]:
%discover

test-esp32  ws://10.39.40.135:8266  
samd51      serial:///dev/ttyACM0   
stm32       serial:///dev/ttyACM1   
test-esp32  serial:///dev/ttyUSB2   
pico        serial:///dev/ttyUSB0   
nrf52840    serial:///dev/ttyACM2   
huzzah32    serial:///dev/ttyUSB1   


The esp32 is available both over USB (serial:///dev/ttyUSB2) and wirelessly via webrepl (ws://10.39.40.135:8266).

In [8]:
%connect test-esp32 ws

for i in range(5): print(i, i**20)

Connected to test-esp32 @ ws://10.39.40.135:8266
0 0
1 1
2 1048576
3 3486784401
4 1099511627776


Connection to a device is maintained until changed. *Hint:* open several notebooks to connect to more than one device simultaneously. 

The `%%connect` cell magic runs code sequentially on one or more processors:

In [8]:
%%connect --all --host

import sys
print("{:18}   ({})".format(sys.platform, sys.implementation.name))


----- HOST

linux                (cpython)

----- test-esp32

esp32                (micropython)

----- samd51

MicroChip SAMD51     (circuitpython)

----- stm32

pyboard              (micropython)

----- test-esp32

esp32                (micropython)

----- pico

esp32                (micropython)

----- nrf52840

nRF52840             (circuitpython)

----- huzzah32

esp32                (micropython)


`%softreset` resets the MicroPython interpreter and has a similar effect to pressing the reset button of the device:

In [8]:
%name

a = 5
print(a)
%softreset
print("this will fail ...")
print(a)

test-esp32
5

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

this will fail ...


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
NameError: name 'a' isn't defined


Consult the documentation for [device configuration](dev_config.ipynb) for automated file upload. 

The kernel has many more features. `%lsmagic` prints a complete list; use help (`-h` flag) on individual magics for documentation and examples.